In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from functools import partial
import random
import keras
from keras.datasets import mnist

In [2]:
class HybridMPSOCNN:
    def __init__(self, input_shape, max_iters_level1, max_iters_level2, alpha, tmax, num_classes):
        self.input_shape = input_shape
        self.max_iters_level1 = max_iters_level1
        self.max_iters_level2 = max_iters_level2
        self.alpha = alpha
        self.tmax = tmax
        self.num_classes = num_classes

    def cnn(self, Pi, Pij, X_train, y_train, X_val, y_val):
      model = Sequential()
      # Pi = (nC, nP, nF)
      nC, nP, nF = [int(x) for x in Pi]
      # Pij = (c_nf, c_fs, c_pp, c_ss, p_fs, p_ss, p_pp, op)
      for i in range(nC):
          c_nf, c_fs, c_pp, c_ss = [int(x) for x in Pij[:4]]
          padding = 'same'
          if i == 0:
              model.add(Conv2D(c_nf, (c_fs, c_fs), strides=(c_ss, c_ss), padding=padding, activation='relu', input_shape=self.input_shape))
          else:
              model.add(Conv2D(c_nf, (c_fs, c_fs), strides=(c_ss, c_ss), padding=padding, activation='relu'))
          padding = 'same'
          if i < nP:
              p_fs, p_ss, p_pp = [int(x) for x in Pij[4:7]]
              p_fs = max(p_fs, 2)
              p_ss = max(p_ss, 2)
              if X_train.shape[1] >= p_fs and X_train.shape[2] >= p_fs:
                  model.add(MaxPooling2D(pool_size=(p_fs, p_fs), strides=(p_ss, p_ss), padding=padding))

      model.add(Flatten())
      for i in range(nF):
          model.add(Dense(int(Pij[7]), activation='relu'))
      model.add(Dense(self.num_classes, activation='softmax'))
      model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1, verbose=1)
      loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
      val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
      print("Train loss: {:.4f}, accuracy: {:.4f}".format(loss, accuracy))
      print("Validation loss: {:.4f}, accuracy: {:.4f}".format(val_loss, val_accuracy))
      fitness = np.max(history.history['val_accuracy'])
      return fitness


    def level2_pso(self, Pi, X_train, y_train, X_val, y_val, search_space):
        m2 = 5 * (Pi[0] + Pi[1])
        tmax2 = self.max_iters_level2
        n_particles = m2

        # Ensure pool size does not exceed input dimensions
        search_space[4, 1] = min(self.input_shape[0], search_space[4, 1])
        search_space[4, 1] = min(self.input_shape[1], search_space[4, 1])

        particles = np.random.uniform(search_space[:, 0], search_space[:, 1], (n_particles, search_space.shape[0])).astype(int)
        pbest = particles.copy()
        gbest = particles[np.argmax([self.cnn(Pi, p, X_train, y_train, X_val, y_val) for p in particles])]
        vel = np.zeros_like(particles)

        for t in range(tmax2):
            for i, particle in enumerate(particles):
                vel = vel.astype('float64')
                particle = particle.astype('float64')
                vel[i] += 2 * np.random.rand() * (pbest[i] - particle) + 2 * np.random.rand() * (gbest - particle)
                vel = np.clip(vel, search_space[:, 0] - particle, search_space[:, 1] - particle)
                particle += vel[i].astype(int)


                new_fitness = self.cnn(Pi, particle, X_train, y_train, X_val, y_val)
                if new_fitness > self.cnn(Pi, pbest[i], X_train, y_train, X_val, y_val):
                    pbest[i] = particle.copy()
                if new_fitness > self.cnn(Pi, gbest, X_train, y_train, X_val, y_val):
                    gbest = particle.copy()

        return gbest

    def level1_pso(self, X_train, y_train, X_val, y_val):
        m1 = 5
        search_space_1 = np.array([(1, 5), (1, 5), (1, 5)], dtype=np.int32)
        search_space_2 = np.array([(1, 64), (1, 7), (0, 1), (1, 3), (1, 7), (1, 3), (0, 1), (1, 1024)], dtype=np.int32)

        c1 = 2
        c2 = 2
        n_particles = m1
        tmax1 = self.max_iters_level1

        particles = np.random.uniform(search_space_1[:, 0], search_space_1[:, 1], (n_particles, search_space_1.shape[0])).astype(int)
        pbest = particles.copy()
        gbest = particles[np.argmax([self.level2_pso(p, X_train, y_train, X_val, y_val, search_space_2) for p in particles])]
        vel = np.zeros_like(particles)

        for t in range(tmax1):
            if t < self.alpha * self.tmax:
                w = 0.9
            else:
                w = 1 / (1 + np.exp(10 * t - 2 * tmax1) / tmax1)

            for i, particle in enumerate(particles):
                vel[i] += c1 * np.random.rand() * (pbest[i] - particle) + c2 * np.random.rand() * (gbest - particle)
                vel = np.clip(vel, search_space_1[:, 0] - particle, search_space_1[:, 1] - particle)
                particle += vel[i].astype(int)

                new_fitness = self.level2_pso(particle, X_train, y_train, X_val, y_val, search_space_2)
                if new_fitness > self.level2_pso(pbest[i], X_train, y_train, X_val, y_val, search_space_2):
                    pbest[i] = particle.copy()
                if new_fitness > self.level2_pso(gbest, X_train, y_train, X_val, y_val, search_space_2):
                    gbest = particle.copy()

        gbest_particle = self.level2_pso(gbest, X_train, y_train, X_val, y_val, search_space_2)
        return gbest, gbest_particle

    def run(self, X_train, y_train, X_val, y_val):
        level1_opt, level2_opt = self.level1_pso(X_train, y_train, X_val, y_val)
        print("Optimal level 1 parameters:", level1_opt)  # [nC, nP, nF]
        print("Optimal level 2 parameters:", level2_opt)  # [c_nf, c_cs, c_pp, c_ss, p_fs, p_ss, p_pp, op]

In [3]:
# Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess the data: normalize and reshape the images
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Convert the labels to one-hot encoded format
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Create an instance of HybridMPSOCNN optimizer
input_shape = (28, 28, 1)
max_iters_level1 = 10
max_iters_level2 = 10
alpha = 0.2
tmax = 10
num_classes = 10

optimizer = HybridMPSOCNN(input_shape, max_iters_level1, max_iters_level2, alpha, tmax, num_classes)

# Run optimization
X_val, y_val = X_test, y_test
optimizer.run(X_train, y_train, X_val, y_val)

1875/1875 [==============================] - 23s 6ms/step - loss: 0.3396 - accuracy: 0.8913 - val_loss: 0.1684 - val_accuracy: 0.9514
Train loss: 0.1608, accuracy: 0.9522
Validation loss: 0.1684, accuracy: 0.9514
1875/1875 [==============================] - 16s 6ms/step - loss: 0.3671 - accuracy: 0.8830 - val_loss: 0.2875 - val_accuracy: 0.9078
Train loss: 0.2733, accuracy: 0.9112
Validation loss: 0.2875, accuracy: 0.9078
1875/1875 [==============================] - 15s 6ms/step - loss: 2.2737 - accuracy: 0.1306 - val_loss: 2.2693 - val_accuracy: 0.1328
Train loss: 2.2692, accuracy: 0.1332
Validation loss: 2.2693, accuracy: 0.1328
1875/1875 [==============================] - 16s 6ms/step - loss: 0.3688 - accuracy: 0.8709 - val_loss: 0.0737 - val_accuracy: 0.9763
Train loss: 0.0778, accuracy: 0.9758
Validation loss: 0.0737, accuracy: 0.9763
1875/1875 [==============================] - 16s 6ms/step - loss: 0.1555 - accuracy: 0.9563 - val_loss: 0.1004 - val_accuracy: 0.9775
Train loss: 0.

KeyboardInterrupt: ignored